We are going to use discrete morse theory to the essential 472 nodes in the complete reduced 10-D space to see whether we can reduce it to the 7 dim space over p53 and then apply the same method again to the 7-D space

In [21]:
import json
from homologyHelpers import *

In [22]:
with open('essentialPNList.dat','r') as file:
    pnlist = json.load(file)

In [23]:
for i in range(len(pnlist)):
    pnlist[i] = [tuple(x) for x in pnlist[i]]

In [24]:
projectedFactor = set([x[0] for x in pnlist])

In [25]:
projectedFactor

{(-2, 0, -1, 1), (0, -2, -1, 1), (0, -1, -2, 1), (0, -1, 1, -2)}

In [26]:
factorStatistics = dict()
for e in projectedFactor:
    factorStatistics[e] = 0

realFactor = 0
for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[realFactor]
    factorStatistics[currFactor]+=1

In [27]:
factorStatistics

{(0, -1, -2, 1): 155,
 (0, -2, -1, 1): 155,
 (-2, 0, -1, 1): 79,
 (0, -1, 1, -2): 83}

In [28]:
factorStatistics = dict()
for e in projectedFactor:
    factorStatistics[e] = set()

realFactor = 0
for i in range(len(pnlist)):
    currPN = pnlist[i]
    currFactor = currPN[realFactor]
    rest = currPN[:realFactor] + currPN[realFactor+1:]
    rest = tuple(rest)
    factorStatistics[currFactor].add(rest)

In [29]:
s1 = factorStatistics[(0, -1, -2, 1)] 
s2 = factorStatistics[(0, -2, -1, 1)]
s3 = factorStatistics[(0, -1, 1, -2)]
s4 = factorStatistics[(-2, 0, -1, 1)]

In [30]:
print(len(s3-s1),len(s3-s2),len(s4-s1),len(s4-s2))

9 9 8 8


In [11]:
subComplexPNList = []
preAttPNList = []
for pn in pnlist:
    if pn[0] not in [(0, -1, -2, 1),(0, -2, -1, 1)]:
        subComplexPNList.append(pn)
    else:
        preAttPNList.append(pn)
len(subComplexPNList),len(preAttPNList)

(162, 310)

In [12]:
subComplexIndex[0]

NameError: name 'subComplexIndex' is not defined

In [13]:
faceLatticeList = []
for f in ['x_o_2_FL.dat','(x+y)z_o_2_FL.dat']:
    with open(f,'r') as file:
        faceLatticeList.append(json.load(file))

In [14]:
for i in range(len(faceLatticeList)):
    tempFaceList = dict()
    currFL = faceLatticeList[i]
    for key in currFL:
        tempFaceList[eval(key)] = [tuple(x) for x in currFL[key]]
    faceLatticeList[i] = tempFaceList

In [15]:
apnList = []
for f in ['x_o_2_PN.json','(x+y)z_o_2_PN.json']:
    apnList.append(loadAtomicChainComplex(f))

In [17]:
len(boundaryMatrix)

8001

In [16]:
# preAtt
preAtt = set()
#for i in range(len(preAttAPNList))
for i in range(1):
    pn = preAttPNList[i]
    pn = [apnList[j][pn[j]]for j in range(len(apnList))]
    temp = tensorProduct(pn)
    for k in temp:
        preAtt|= temp[k]


# boundary matrix
boundaryMatrix = dict()
coboundaryMatrix = dict()

# construct boundary matrix of subComplex
#for i in range(len(subComplexPNList)):
for i in range(1):
    pn = subComplexPNList[i]
    pn = [apnList[j][pn[j]]for j in range(len(apnList))]
    allFaces = set()
    pnFL = tensorProduct(pn)
    for k in pnFL:
        allFaces|= pnFL[k]
    
    for face in allFaces:
        if face in boundaryMatrix:
            continue
        
        boundary = set(product([face[0]], faceLatticeList[1][face[1]]))
        boundary |= set(product(faceLatticeList[0][face[0]],[face[1]]))
        
        boundaryMatrix[face] = boundary 
        
# # construct coboundary matrix of subComplex
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

# construct the subComplex
subComplex = dict()
for cell in boundaryMatrix:
    if cell in preAtt:
        continue
    
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

# discrete morse graph

queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)
print('queens: {}, kings: {}, aces: {}'.format(len(queens),len(kings),len(aces)))

queens: 3933, kings: 3933, aces: 0


In [18]:
def checkCoreduction(face, coboundary, currComplex,used):
    for cell in coboundary:        
        if len(currComplex[cell] - used) == 1:
            return True
    return False


# used is the preset attractors
def genMatchingWithPreAtt(currComplex,coboundaryMatrix,used):
    
    kings = []
    queens = []
    aces = []
    
    count = 0
    while currComplex:
        count +=1
        
        # find queen, king pairs
        flag = True
        
        # flag: find coreducible pair in previous around
        while flag:
            tempComplex = dict()
            flag = False
            for cell in currComplex:

                tempDiff = currComplex[cell] - used

                if len(tempDiff) == 1:
                    queen = list(tempDiff)[0]
                    king = cell

                    queens.append(queen)
                    kings.append(king)

                    used.add(king)
                    used.add(queen)

                    flag = True

                if cell not in used:
                    tempComplex[cell] = tempDiff
            
            currComplex = tempComplex
            
        #print(queens, kings, aces)
        #print(tempComplex)
        
        # check coreducible or not
        flag = False # exist coreducible or not
        while not flag:
            tempComplex = dict()
            
            for key in currComplex:
                
                # if already have coreduction in previous loop, then only restore the rest of dictionary
                if flag:
                    tempComplex[key] = currComplex[key]
                    continue
                
                
                if len(currComplex[key]) == 0:
                    aces.append(key)
                    used.add(key)

                    # if key is a top cell, just continue
                    if key not in coboundaryMatrix:
                        continue

                    # if there is a coreducible pair we break the aces collection process
                    if checkCoreduction(key, coboundaryMatrix[key], currComplex, used):
                        flag = True
                        
                else:
                    tempComplex[key] = currComplex[key]
            
            currComplex = tempComplex
            
            # if empty
            if not currComplex:
                break
        
    return queens, kings, aces

def isFace(f,c):
    if set(f).issubset(set(c)):
        return True
    
    return False

def getBoundaryMatrix(apnList):
    faceLattice = dict()
    for index in apnList:
        pn = apnList[index]
        d = len(pn)
        for dim in range(d-1,0,-1):
            cells = pn[dim]
            faces = pn[dim-1]
            for c in cells:
                for f in faces:
                    if isFace(f,c):
                        sf = tuple(sorted(f))
                        sc = tuple(sorted(c))
                        if sc not in faceLattice:
                            faceLattice[sc] = []
                        faceLattice[sc].append(sf)
        
        # the zero diminsion
        cells = pn[0]
        for point in cells:
            faceLattice[tuple(sorted(point))] = []
            
            
    # remove the duplicate faces
    for key in faceLattice:
        faceLattice[key] = list(set(faceLattice[key]))
            
    return faceLattice

def getCoboundaryMatrix(boundarMatrix):
    coboundaryMatrix = dict()
    
    for cell in boundaryMatrix:
        boundary = boundaryMatrix[cell]
        for b in boundary:
            if b not in coboundaryMatrix:
                coboundaryMatrix[b] = []
            coboundaryMatrix[b].append(cell)
    return coboundaryMatrix

   

In [19]:
pn = [apnList[j][pn[j]]for j in range(len(apnList))]

TypeError: unhashable type: 'dict'

In [20]:
temp = tensorProduct([apnList[0][subComplexIndex[0][0]],apnList[1][subComplexIndex[0][1]]])
sum([len(temp[k]) for k in temp])

NameError: name 'subComplexIndex' is not defined

In [32]:
apnList[0][subComplexIndex[0][0]]

{0: {(11,), (12,), (15,), (16,), (18,), (21,)},
 1: {(11, 16),
  (11, 18),
  (11, 21),
  (12, 15),
  (12, 16),
  (12, 18),
  (15, 18),
  (15, 21),
  (16, 21)},
 2: {(11, 12, 16, 18),
  (11, 15, 18, 21),
  (11, 16, 21),
  (12, 15, 16, 21),
  (12, 15, 18)},
 3: {(11, 12, 15, 16, 18, 21)}}

In [111]:
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

NameError: name 'factorAPN' is not defined

In [ ]:
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

In [19]:
print('queens: {}, kings: {}, aces: {}'.format(len(queens),len(kings),len(aces)))

NameError: name 'queens' is not defined

# test the computation

In [31]:
import json
from homologyHelpers import *

In [33]:
if __name__ == "__main__":
    
    with open('essentialPNList.dat','r') as file:
        pnlist = json.load(file)

    for i in range(len(pnlist)):
        pnlist[i] = [tuple(x) for x in pnlist[i]]

    # pnList for subComplex and preAtt
    subComplexPNList = []
    preAttPNList = []
    
    for pn in pnlist:
        if pn[0] not in [(0, -1, -2, 1),(0, -2, -1, 1)]:
            subComplexPNList.append(pn)
        else:
            preAttPNList.append(pn)

    # face lattice list
    faceLatticeList = []
    for f in ['x_o_2_FL.dat','(x+y)z_o_2_FL.dat']:
        with open(f,'r') as file:
            faceLatticeList.append(json.load(file))

    for i in range(len(faceLatticeList)):
        tempFaceList = dict()
        currFL = faceLatticeList[i]
        for key in currFL:
            tempFaceList[eval(key)] = [tuple(x) for x in currFL[key]]
        faceLatticeList[i] = tempFaceList

    # apnList
    apnList = []
    for f in ['x_o_2_PN.json','(x+y)z_o_2_PN.json']:
        apnList.append(loadAtomicChainComplex(f))


    # preAtt
    preAtt = set()
    for i in range(len(preAttPNList)):
    #for i in range(1):
        pn = preAttPNList[i]
        pn = [apnList[j][pn[j]]for j in range(len(apnList))]
        temp = tensorProduct(pn)
        for k in temp:
            preAtt|= temp[k]

    # boundary matrix
    boundaryMatrix = dict()
    coboundaryMatrix = dict()

    # construct boundary matrix of subComplex
    for i in range(len(subComplexPNList)):
    #for i in range(1):
        pn = subComplexPNList[i]
        pn = [apnList[j][pn[j]]for j in range(len(apnList))]
        allFaces = set()
        pnFL = tensorProduct(pn)
        for k in pnFL:
            allFaces|= pnFL[k]

        for face in allFaces:
            if face in boundaryMatrix:
                continue

            boundary = set(product([face[0]], faceLatticeList[1][face[1]]))
            boundary |= set(product(faceLatticeList[0][face[0]],[face[1]]))

            boundaryMatrix[face] = boundary 

    # # construct coboundary matrix of subComplex
    coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

    # construct the subComplex
    subComplex = dict()
    for cell in boundaryMatrix:
        if cell in preAtt:
            continue

        subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

    # discrete morse graph

    queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)
    print('queens: {}, kings: {}, aces: {}'.format(len(queens),len(kings),len(aces)))

queens: 257661, kings: 257661, aces: 0


In [34]:
len(preAtt)

1466943

In [35]:
queens[:10]

[((15,), (208,)),
 ((21,), (74,)),
 ((21,), (414,)),
 ((21,), (69,)),
 ((11, 21), (414,)),
 ((15,), (239,)),
 ((21,), (227,)),
 ((21,), (229,)),
 ((21,), (208,)),
 ((21,), (141,))]

In [36]:
kings[:10]

[((12, 15), (208,)),
 ((11, 21), (74,)),
 ((16, 21), (414,)),
 ((16, 21), (69,)),
 ((11, 16, 21), (414,)),
 ((12, 15), (239,)),
 ((11, 21), (227,)),
 ((11, 21), (229,)),
 ((15, 21), (208,)),
 ((16, 21), (141,))]

In [37]:
len(s1)

155

In [38]:
s1

{((0, -2, -1, 1, 2, 3, 4, 5, 6, 7),),
 ((0, -2, 2, -1, 1, 3, 4, 5, 6, 7),),
 ((0, -2, 2, 4, -1, 1, 3, 5, 6, 7),),
 ((0, -2, 2, 4, 6, -1, 1, 3, 5, 7),),
 ((0, -2, 4, -1, 1, 2, 3, 5, 6, 7),),
 ((0, -1, -2, 1, 2, 3, 4, 5, 6, 7),),
 ((0, -1, 1, -2, 2, 3, 4, 5, 6, 7),),
 ((0, -1, 1, 2, -2, 3, 4, 5, 6, 7),),
 ((0, -1, 1, 2, 3, -2, 4, 5, 6, 7),),
 ((0, -1, 1, 2, 3, 4, -2, 5, 6, 7),),
 ((0, -1, 1, 2, 3, 4, 5, -2, 6, 7),),
 ((0, -1, 1, 2, 4, -2, 3, 5, 6, 7),),
 ((0, -1, 1, 2, 4, 5, -2, 3, 6, 7),),
 ((0, -1, 1, 4, -2, 2, 3, 5, 6, 7),),
 ((0, -1, 1, 4, 5, -2, 2, 3, 6, 7),),
 ((0, -1, 2, -2, 1, 3, 4, 5, 6, 7),),
 ((0, -1, 2, 4, -2, 1, 3, 5, 6, 7),),
 ((0, -1, 4, -2, 1, 2, 3, 5, 6, 7),),
 ((0, 1, -2, -1, 2, 3, 4, 5, 6, 7),),
 ((0, 1, -2, 2, -1, 3, 4, 5, 6, 7),),
 ((0, 1, -2, 2, 3, -1, 4, 5, 6, 7),),
 ((0, 1, -2, 2, 3, 4, -1, 5, 6, 7),),
 ((0, 1, -2, 2, 3, 4, 5, -1, 6, 7),),
 ((0, 1, -2, 2, 3, 4, 5, 6, -1, 7),),
 ((0, 1, -2, 2, 3, 4, 6, -1, 5, 7),),
 ((0, 1, -2, 2, 4, -1, 3, 5, 6, 7),),
 ((0, 1, -2,

# for the rest p53

In [39]:
apnList = loadAtomicChainComplex('(x+y)z_o_2_PN.json')
factorAPN = dict()
for k in s1:
    factorAPN[k[0]] = apnList[k[0]]

In [40]:
boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

In [41]:
preAtt = set()
tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

In [42]:
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)
print(len(queens),len(kings),len(aces))

14418 14418 1


In [43]:
aces

[(465,)]